In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from IPython.display import display
import custom

In [ ]:
# age: continuous.
# workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
# fnlwgt: continuous.
# education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
# education-num: continuous.
# marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
# occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
# relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
# race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
# sex: Female, Male.
# capital-gain: continuous.
# capital-loss: continuous.
# hours-per-week: continuous.
# native-country

In [2]:
headers = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
original_data = pd.read_csv("adult.data.csv", header = None)
original_data.columns = headers
data = original_data.drop(["fnlwgt", "capital-gain","capital-loss","native-country"], axis = 1)

data = custom.one_hot_encoding(data, ['workclass', 'education' ,'marital-status', 'occupation', 'relationship', 'race', 'sex', 'income'])

data.fillna(data.mean())

data = custom.del_anomaly(data, ['age', 'education-num', 'hours-per-week'], 2)

data = custom.scaling(data, ['age', 'education-num', 'hours-per-week'])

display(data)


,age,education-num,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,...,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,income_ <=50K,income_ >50K
0,0.458333,0.777778,0.489362,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,True,False
2,0.437500,0.333333,0.489362,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,True,False
3,0.750000,0.111111,0.489362,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,True,False
4,0.229167,0.777778,0.489362,False,False,False,False,True,False,False,...,True,False,False,True,False,False,True,False,True,False
5,0.416667,0.888889,0.489362,False,False,False,False,True,False,False,...,True,False,False,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.208333,0.666667,0.446809,False,False,False,False,True,False,False,...,True,False,False,False,False,True,True,False,True,False
32557,0.479167,0.333333,0.489362,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,True
32558,0.854167,0.333333,0.489362,False,False,False,False,True,False,False,...,False,False,False,False,False,True,True,False,True,False
32559,0.104167,0.333333,0.063830,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,True,False


In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

x = data.iloc[:,:-2]
t = data.iloc[:,-2:]

tensor_x = torch.tensor(x.values.tolist(), dtype = torch.float, device = device)
tensor_t = torch.tensor(t.values.tolist(), dtype = torch.long, device = device)
tensor_t = torch.argmax(tensor_t, dim = 1)
train = list(zip(tensor_x,tensor_t)) #zip : 리스트 두개를 튜플로 묶어주는 함수 (텐서도 가능)
batch = DataLoader(train, batch_size = 100, shuffle = True) #DataLoader : 자동으로 batch를 나눠주는 함수

F = nn.Sequential(
    nn.Linear(x.shape[1], 16, device = device),
    nn.Sigmoid(),
    nn.Linear(16,8, device = device),
    nn.Sigmoid(),
    nn.Linear(8,2, device = device),
    nn.Sigmoid(),
    #nn.LogSoftmax(),
)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 0.5)
epoch = 100


for e in range(epoch) :
    loss_sum = 0
    for x, t in batch :
        y = F(x)
        loss = loss_function(y, t)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_sum += loss
    if (e+1) % 10 == 0 :
        print("epoch {} | loss {}".format(e+1,loss_sum/len(batch)))
        


epoch 10 | loss 0.5629591345787048
epoch 20 | loss 0.5625405311584473
epoch 30 | loss 0.5546534657478333
epoch 40 | loss 0.4773169159889221
epoch 50 | loss 0.4725128412246704
epoch 60 | loss 0.47162333130836487
epoch 70 | loss 0.4711349308490753
epoch 80 | loss 0.470445454120636
epoch 90 | loss 0.4704441726207733
epoch 100 | loss 0.470001220703125


In [13]:
validation_data = pd.read_csv("adult.test.csv", header = None)
validation_data.columns = headers

validation_data = validation_data.drop(["fnlwgt", "capital-gain","capital-loss","native-country"], axis = 1)

validation_data = custom.one_hot_encoding(validation_data, ['workclass', 'education' ,'marital-status', 'occupation', 'relationship', 'race', 'sex', 'income'])

validation_data.fillna(validation_data.mean())

#validation_data = custom.del_anomaly(validation_data, ['age', 'education-num', 'hours-per-week'], 2)

validation_data = custom.scaling(validation_data, ['age', 'education-num', 'hours-per-week'], original_data)

display(validation_data)




,age,education-num,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,...,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,income_ <=50K.,income_ >50K.
0,0.109589,0.400000,0.397959,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,True,False
1,0.287671,0.533333,0.500000,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,True,False
2,0.150685,0.733333,0.397959,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,False,True
3,0.369863,0.600000,0.397959,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,True
4,0.013699,0.600000,0.295918,True,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,0.301370,0.800000,0.357143,False,False,False,False,True,False,False,...,False,False,False,False,False,True,True,False,True,False
16277,0.643836,0.533333,0.397959,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,True,False
16278,0.287671,0.800000,0.500000,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,True,False
16279,0.369863,0.800000,0.397959,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,True,True,False


In [16]:
val_x = validation_data.iloc[:,:-2]
val_t = validation_data.iloc[:,-2:]

val_x = torch.tensor(val_x.values.tolist(), dtype = torch.float)
val_t = torch.tensor(val_t.values.tolist(), dtype = torch.long)
val_t = torch.argmax(val_t, dim = 1)

cnt = 0
for i in range(val_x.shape[0]) :
    val_y = F(val_x[i])
    output = torch.argmax(val_y)
    if output == val_t[i] :
        cnt += 1

print(cnt / val_x.shape[0])

torch.save(F,"income.pt")

0.8326884098028376


In [4]:
a = torch.tensor([-2,0,1,3], dtype = torch.float)

b = nn.functional.log_softmax(a)

c = nn.functional.log_softmax(nn.functional.log_softmax(a))


print(a)
print(b)
print(c)

tensor([-2.,  0.,  1.,  3.])
tensor([-5.1755, -3.1755, -2.1755, -0.1755])
tensor([-5.1755, -3.1755, -2.1755, -0.1755])


C:\Users\jinsa\AppData\Local\Temp\ipykernel_17948\3320495553.py:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  b = nn.functional.log_softmax(a)
C:\Users\jinsa\AppData\Local\Temp\ipykernel_17948\3320495553.py:5: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  c = nn.functional.log_softmax(nn.functional.log_softmax(a))
